This is a model for exploratory analysis based on a tutorial from Giraffa, Iona Barbos.
The methodology is complete and logical in progression.

The data is based upon casino customer gaming manetary transactions. There are two levels of transactions, each
with a deposit and a withdrawal. L1 transactions are customer deposits to/withdrawal from their gambling account.L2 transactions are deposits to/withdrawals from the customer gaming account for gambling activities

Load libraries

In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from datetime import datetime as dt
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio


Tidy up data appearance

In [ ]:
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.max_columns', 999)

Import the csv file

In [ ]:
df = pd.read_csv('Online_casino_DIB.csv')
df.head()

Examine the shape, datatype and info for the df

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe(exclude='float64')

ReqTimeUTC is type obj so convert it to date time
ReqTimeUTC has a number of digits in the entry for the time. If we are not going to use time you can clean up the entry by stripping of the time component of the entry - see the code line in the markdown
ReqTimeUTC has date/time entries beyond 2020-02-29 so exclude those values
Data is being excluded so create new df

In [ ]:
df_raw = df[df.ReqTimeUTC <= '2020-02-29 00:00:00+00:00']\
    .copy()

In [ ]:
df_raw['ReqTimeUTC'] = pd.to_datetime(df_raw['ReqTimeUTC'])

If you don't want the time elements for future use you can remove them using the following code line

df_raw['ReqTimeUTC'] = df_raw['ReqTimeUTC'].dt.strftime('%Y-%m-%d')

In [ ]:
df_raw.head()

Check the Transaction type column
 Transactions labeled as “LOYALTYCARDDEBIT” are Level 2 deposits. 
 Transactions labeled as “LOYALTYCARDCREDIT” are Level 2 withdrawals. 
 Transactions labeled as “LOYALTYCARDCREDITCL” or “LOYALTYCARDCREDITACH” are Level 1 deposits 
 made via a card or ACH respectively.

Map
 LOYALTYCARDDEBIT = L2D
 LOYALTYCARDCREDIT = L2W
 LOYALTYCARDCREDITCL = L1D

In [ ]:
df_raw.describe(exclude='float64')

In [ ]:
df_raw['TransactionType'].unique()

Map the shortforms into the df column

In [ ]:
df_raw.TransactionType = df_raw.TransactionType\
    .map({'LOYALTYCARDDEBIT':'L2D','LOYALTYCARDCREDITCL':'L1D',
          'LOYALTYCARDCREDIT':'L2W'})

df_raw.head()

Simplify the AccountIdentifier column by stripping off 'Customer' with regex

In [ ]:
df_raw['AccountIdentifier'] = df_raw['AccountIdentifier']\
    .str.replace(r'^customer','',
                                                                      regex=True)
df_raw.head()

Rename some of the columns to more useable form

In [ ]:
df_raw.rename(columns= {'AccountIdentifier':'user', 'ReqTimeUTC':'date',
                        'TransactionAmount':'amount', 
                        'TransactionType':'type',
                        'Status':'status'},inplace=True)

df_raw.head()

Group data by users by user and type 
Create df for user type and define - user_type_
Plot

In [ ]:
df_raw.groupby(['user', 'type']).count().reset_index()

In [ ]:
user_type_ = df_raw.groupby(['user', 'type']).count()\
    .reset_index()

In [ ]:
sns.histplot(data=user_type_, x='date', bins=50, 
             hue='type',)

Histplot even with hue is not very informative

In [ ]:
sns.histplot(data=user_type_, x='date', hue='type',
             cumulative=True, stat='density', 
             element='step', fill=False)

The cumulative version of the histplot is more informative.
L2D activities closely follows LD1 activities
There is more separation between the L#D activities and the L2W activities

In further analysis we will drop the L1D activities and keep on the L2D

In [ ]:
df_working = df_raw.copy()
df_working.head(1)

In [ ]:
df_working.columns

In [ ]:
df_working = df_working[(df_working.type == "L2D") & \
    (df_working.status == "APPROVED")].reset_index(drop=True)

df_working.shape

In [ ]:
df_working = df_working[['user', 'date', 'type', 'amount']]

df_working = df_working.sort_values(['user', 'date'])\
    .reset_index(drop=True)

df_working.head()

Check to see if the patrons who make the most frequent deposits also the patrons who deposit the most money.
Look at the data for the top 20 depositors
1. Get a frequency count
2. Reset the index
3. Select the first 2 columns
4. Sort by date, reversee the sort and take the top 20 values
5. Put the top 20 in a new df and clean it up

In [ ]:
df_working.groupby('user').count()

In [ ]:
df_working.groupby('user').count().reset_index()

In [ ]:
df_working.groupby('user').count().reset_index() \
    .iloc[:,0:2]

In [ ]:
df_working.groupby('user').count().reset_index() \
    .iloc[:,0:2].sort_values("date")[::-1][0:20]

In [ ]:
df_top20_freq = df_working.groupby('user').count()\
    .reset_index()\
    .iloc[:,0:2].sort_values("date")[::-1][0:20]

Perform a similar activity for the top spenders

In [ ]:
df_top20_spenders = df_working[['user', 'amount' ]] \
    .groupby('user').sum().reset_index()\
        .sort_values('amount') \
        [::-1][0:20]

In [ ]:
df_top20_spenders.head()

Plot Top 20 side-by-side (ncols=2)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15, 5))

sns.barplot(df_top20_freq, x='date', y='user', 
            ax = ax[0])
sns.barplot(df_top20_spenders, x='amount', 
            y='user', ax=ax[1])

The plot shows that the most frequent patrons are not the same as the patrons who deposit the most money


Work the date column to see what can extracted from it
Extract the hour from the date column
Extract the day of the week from the date column
Create new columns in the df for each of these elements
The day of the week is initiall presented as integers 0-6 Mon-Sun
Create dictionary to convert day of the week from int to day name and map to df

In [ ]:
df_working.date.dt.hour
df_working['hour'] = df_working.date.dt.hour

In [ ]:
df_working.date.dt.day_of_week
df_working['day_of_week'] = df_working.date.dt\
    .day_of_week
df_working

In [ ]:
day_of_the_week_dict = {0: 'Monday',
                        1: 'Tuesday',
                        2: 'Wednesday',
                        3: 'Thursday',
                        4: 'Friday',
                        5: 'Saturday',
                        6: 'Sunday',
                        }

In [ ]:
df_working['day_of_week'] = df_working['day_of_week']\
    .map(day_of_the_week_dict)
df_working

Using the day and time of day data are there any patterns that are evident by plotting these data.
Create a heat map for this
Creating a heat map requires some manipulation of the data into a 'confusion matrix' where the day of the week is the index with the hours of the day as the columns of the matrix
Use the pivot table method from pandas to create the confusion matrix from the df we created

In [ ]:
df_working[['hour', 'day_of_week', 'type']]\
    .groupby(['hour', 'day_of_week']).count().reset_index()

In [ ]:
h_d_g = df_working[['hour', 'day_of_week', 'type']]\
    .groupby(['hour', 'day_of_week']).count().reset_index()
    
h_d_g

In [ ]:
df_heatmap = pd.pivot_table(h_d_g, values='type', index='day_of_week',
                            columns='hour')

In [ ]:
sns.heatmap(df_heatmap)

Examine daily activity
How patrons made deposits in a day 
and what were the values
Plot the results

In [ ]:
df_working.columns

In [ ]:
daily_activity = df_working.groupby(['user', 'date']) \
    .agg({'amount': 'sum', 'type': 'count'}) \
        .reset_index()
        
daily_activity

In [ ]:
daily_activity[daily_activity.user=="customer1"][0:20]

daily_activity

In [ ]:
c1 = daily_activity[daily_activity.user == '1'][0:20]
c1

looking at customer 1 there are gaps in the time line 
where this customer was not active
This gap will not let us plot the data
Create new df for customer 1 and fill in the gaps

In [ ]:
user_1_df = daily_activity[daily_activity.user == '1']
user_1_df

In [ ]:
full_customer_df = pd.DataFrame()

for customer_id in daily_activity.user.unique():
    customer_df = daily_activity[daily_activity\
        .user == customer_id]
    
    full_range = pd.date_range(customer_df.date.min(),
                           customer_df.date.max(), freq='D')
    
    customer_df = customer_df.set_index(keys = 'date')
    
    customer_df = customer_df.reindex(list(full_range), 
                                      fill_value=0)
    customer_df.user = [customer_id]*len(customer_df)
    
    customer_df = customer_df.reset_index()
    full_customer_df = pd.concat([full_customer_df, customer_df])
    
full_customer_df = full_customer_df.reset_index(drop=True)

In [ ]:
full_customer_df

Work up a Sankey plot

In [ ]:
full_customer_df['month'] = full_customer_df.date.dt.\
    to_period('M')

customer_month = full_customer_df.groupby(['month', 'user']) \
    .count().reset_index().iloc[:, :2]
    
start_ = customer_month.groupby('user').min().reset_index()

end_ = customer_month.groupby('user').max().reset_index()

start_end_df = pd.merge(start_, end_, on='user')

start_end_df


In [ ]:
source_target_value = start_end_df.groupby(['month_x', 'month_y']) \
    .count().reset_index()

In [ ]:
source_target_value

Use LabelEncoder to get data into shape for Sankey plot

In [ ]:
le = LabelEncoder()
le.fit_transform(source_target_value.month_x.unique())

In [ ]:
fig = go.Figure(data=[go.Sankey(node=dict(
    pad=100,
    thickness = 10,
    line = dict(color = 'gray', width = 0.5),
    label = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
            'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb' ],
    color = 'orange'),
                  link = dict(
                      source = le.transform(source_target_value.month_x.tolist()),
                      target = le.transform(source_target_value.month_y.tolist()),
                      value = source_target_value.user.tolist(),
                      hovercolor=['black']
))])

fig.update_layout(title_text="Customer Lifetime: Mar - Feb",
                  font_size=10)
fig.show()